# Week 3 assignment - Toronto Neighbourhoods

At first, load all the libraries

In [2]:
# import statements
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import urllib.request
#beautiful soup
from bs4 import BeautifulSoup

Next, I'm getting the source code (html) of the wikipedia article and save it locally

In [18]:
src_data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# get html source code
req = urllib.request.urlopen(src_data_url)
article = req.read().decode()

with open('List_of_postal_codes_of_Canada:_M', 'w') as fo:
    fo.write(article)


Using the local copy of the wikipedia article, I extract the information from the table and save it to a (local) csv file. 

The csv file uses a semicolon as a delimiter.

In [11]:
article = open('List_of_postal_codes_of_Canada:_M').read()
soup = BeautifulSoup(article, 'html.parser')
tables = soup.find_all('table', class_='sortable')

# get table headings
for table in tables:
    ths = table.find_all('th')
    headings = [th.text.strip() for th in ths]
    if headings[:3] == ['Postcode', 'Borough', 'Neighbourhood']:
        break
    

csv_file = 'Toronto_PO_Codes.csv'
print('Saving data to %s' % csv_file)

with open(csv_file, 'w') as fo:
    print(';'.join(['Postcode', 'Borough', 'Neighbourhood']),file=fo)

print('Wrote headers to %s' %csv_file)

# get column data and write to csv file
with open(csv_file, 'a') as fo:
    lines = 0
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        code,borough,neighbourhood = [td.text.strip() for td in tds[:3]]
        if 'Not assigned' in borough:
            continue
        if  'Not assigned' in neighbourhood:
            neighbourhood = borough
            
        print('; '.join([code,borough,neighbourhood]), file=fo)
        lines = lines+1
        
    
print("Wrote %i lines to %s" % (lines, csv_file))

Saving data to Toronto_PO_Codes.csv
Wrote headers to Toronto_PO_Codes.csv
Wrote 212 lines to Toronto_PO_Codes.csv


Now that the data has been saved to csv,it will be loaded into a panda data frame

In [17]:
df = pd.read_csv(csv_file,sep=';')
df.head(5)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Display the requested shape information

In [21]:
df.shape

(212, 3)